In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import mapping
import folium
import gmaps
import os
import requests
import shutil

In [2]:
with open('../resources/key.txt') as f:
    token = f.read()
    
gmaps.configure(api_key = token)

In [40]:
'''
Set local I/O
'''

# Set folder and filename for rail GIS file
rail_folder = '../data/railways/global/AUS/'
rail_file = rail_folder+'AUS.shp'

# Set folder and filename for rail CSV file
output_folder = '../data/output_images/Australia/'
output_csv = output_folder+'AUS.csv'

# Set folder for new images
image_folder = output_folder+'set_3/'

### Import Routes

In [47]:
routes = gpd.read_file(rail_file)
# routes = routes.to_crs({'init' :'epsg:4269'})
routes.head()

,Line,geometry
0,2654260,"LINESTRING (151.1964198 -33.868837, 151.196246..."
1,2761538,"LINESTRING (151.2010021 -33.8791125, 151.20067..."
2,2949708,"LINESTRING (151.2072229 -33.8803365, 151.20715..."
3,3726276,"LINESTRING (151.1928863 -33.8949456, 151.19359..."
4,3999468,"LINESTRING (150.3013847 -33.7289792, 150.30188..."


In [42]:
routes.shape

(41896, 2)

In [43]:
'''
Get all points
'''

points = []
rows = routes.index.tolist()

geom_types = routes['geometry'].tolist()
for i in rows:
    try:
        route = routes.loc[i]
        if route.geometry.geom_type == 'LineString':
            g = route.geometry
            sec_points = mapping(g)["coordinates"]
            for j in sec_points:
                tmp = (j[0],j[1])
                points.append(tmp)
        elif route.geometry.geom_type == 'MultiLineString':
            g = route.geometry
            tmp = mapping(g)["coordinates"]
            for sec_points in tmp:
                for j in sec_points:
                    tmp = (j[0],j[1])
                    points.append(tmp)
    except Exception as e:
        print("Skipped route number",i,'because',e)        
        
    
print(len(points),'points')   

557284 points


In [26]:
'''
Get points for 1 or more routes
'''

rows = [0]

points = []
for i in range(0,len(routes.OBJECTID.tolist())):
    if i in rows:
    #     print('Route',i)
        try:
            route = routes.iloc[i]
            g = [m for m in route.geometry]
    #         print('\t',len(g),'sections')
            for j in range(0,len(g)):
    #             print('   section',j)
                sec_points = mapping(g[j])["coordinates"]
                for k in range(0,len(sec_points)):
    #                 print('      point',k)
                    points.append(sec_points[k])
        except Exception as e:
            print("Skipped route number",i,'because',e)

print(len(points),'points')

Skipped route number 0 because 'LineString' object is not iterable
0 points


In [ ]:
'''
Get points for single route/section
'''

row = 0
section = 0

points = []
try:
    route = routes.iloc[row]
    g = [i for i in route.geometry]
    len(g)

    points = mapping(g[section])["coordinates"]
except Exception as e:
    print("Skipped route number",row,'because',e)
    
print(len(points),'points')

##### Get nth number

In [44]:
'''
Open csv with current points
'''

curr_df = pd.read_csv(output_csv)
curr_df.head()

,Name,Longitude,Latitude,Catenary
0,151.1964198_-33.868837,151.196420,-33.868837,0
1,151.2070754_-33.8801401,151.207075,-33.880140,0
2,151.9480466_-27.573679499999997,151.948047,-27.573679,1
3,153.0118785_-27.5267477,153.011878,-27.526748,1
4,153.00315669999998_-27.5259447,153.003157,-27.525945,1


In [57]:
'''
Get subselection of points excuding current points
'''
rows = curr_df.index.tolist()
curr_points = []
for row in rows:
    tmp = curr_df.iloc[row].Name
    space = tmp.find('_') 
    longitude = tmp[:space]
    latitude = tmp[space+1:]
    curr_points.append((longitude,latitude))

route_points = []
for point in range(0,len(points),3500):
    tmp = points[point]
    if tmp not in curr_points:
        route_points.append(tmp)

print(len(route_points))

160


In [58]:
'''
Create dataframe
'''

df = pd.DataFrame(route_points, columns=['Longitude','Latitude'])

tmp = list(df.Latitude.tolist())
names = []
for i in range(0,len(tmp)):
    name = str(df.iloc[i].Longitude)+'_'+str(df.iloc[i].Latitude)
    names.append(name)

df['Name'] = names

columns = ['Name','Longitude','Latitude']
df= df[columns]

df.head()

,Name,Longitude,Latitude
0,151.1964198_-33.868837,151.196420,-33.868837
1,144.9294889_-37.739057200000005,144.929489,-37.739057
2,153.3482645_-27.9870979,153.348264,-27.987098
3,174.8435015_-41.1434534,174.843502,-41.143453
4,153.0148648_-27.4721499,153.014865,-27.472150


In [59]:
'''
Plot markers
'''

rows = df.index.tolist()

marker_points = []
for row in rows:
    marker_points.append((df.iloc[row].Latitude,df.iloc[row].Longitude))
    
# Set map centerpoint
coords = marker_points[0]
       
# Define map
m = folium.Map(
    location = coords,
    zoom_start = 13
)
  
# Add points    
for mp in marker_points:
    folium.Marker(mp).add_to(m)
m

In [108]:
'''
Get satellite preview for image
'''

row = 0

figure_layout = {
    'width': '100%',
    'height': '800px'
}
fig = gmaps.figure(center = (df.iloc[row].Latitude,df.iloc[row].Longitude), zoom_level = 19,map_type='SATELLITE',layout=figure_layout)
fig

Figure(layout=FigureLayout(height='800px', width='100%'))

In [60]:
'''
Export points
'''

# Update
with open(output_csv, 'a') as f:
    df.to_csv(f, header=False,index=None)

# Write new
# df.to_csv(output_csv,header=True,index=None)
# df.to_csv('tmp.csv',header=True,index=None)

In [62]:
'''
Get all images 
'''

root = os.path.dirname(os.path.abspath('Collection.ipynb'))
img_folder = root[:-9]+image_folder[3:]
img_folder

url = 'https://maps.googleapis.com/maps/api/staticmap?'
rows = df.index.tolist()
for i in range(0,3):
    row = df.iloc[i]
    center = str(row.Latitude)+','+str(row.Longitude)
    payload = {
        'center': center, 
        'zoom': 20,
        'size': '640x640',
        'scale': 2,
        'format': 'png32',
        'maptype': 'satellite',
        'key': token
    }
    
    r = requests.get(url,params=payload,stream=True)

    name = df.iloc[i]['Name']
    filename = img_folder+name+'.png'
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f) 
    else:
        print(r.status_code)